# Random Forest

The following notebook outlines the model that we have used for training our predictive model for the stations. It should be stated, as it is in the document, that we opted to train a model for each station and for each day. Our decision for this was due to performance. We observed that a model for all stations was very inaccurate. Likewise, there was a great deal of deviation between different datys - this was not just isolted to a weekday:weekend split either. 

Based on this we trained a model for each station and for each day. The result was pickled and stored. Below is an example:
1. **Station:** GRAND CANAL DOCK
2. **Day:** Thursday

We first cleaned the data and created bins on some of the categorical data. We then trained the model and tested on a test set. We also used cross-validation in order to truely assess the performance of the model. Before pickling we have also printed out a .PNG of the tree. This is also attacted in the suporting documentation. 

It is worth noting that we tried a Linear Regression model but found it to be inaccurate when compared to the Random Forest. We also recognise that the volume of data on each station is still sparce. This results in single irregularites having a large effect on our model. For example, there is are two fridays in early March that are vastly different to that of other Fridatys. Considering that this is 25% of our data the effect it has on our prediction is significant. We hypothesis that irregularitied such as this will mean out over time as the dataset grows. However, if this is a trend the this will be built in the model.

Finally, we are working off data that has been collected in late Winter and Spring. We are cognitive of the possibilty that we will soon be predicting the available bikes based off a different season. For example, the effect of a 'sunny day' on the bike availabilty during the winter is significant. However, during the summer, when the weather is more frequently sunny, the weight it has on availabilty may not be so large. This is merely a hypothesis and although we have not adopted this into our model, this would have to be built-in in future itterations. 

In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import export_graphviz
import pydot

import pickle
import math

In [2]:
# Reading in the csv
# Investigating the feature types
df = pd.read_csv('full_dataframe.csv', index_col=0)
df.dtypes

/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


number                int64
name                 object
status               object
bike_stands           int64
available_bikes       int64
available_stands      int64
last_update          object
banking               int64
weather_id            int64
main                 object
description          object
temperature         float64
humidity            float64
wind_speed          float64
dtype: object

In [3]:
# Investigating the 'description' weather feature counts
df['description'].value_counts()

broken clouds                   332210
light rain                      195687
scattered clouds                141655
few clouds                      120134
moderate rain                    79911
clear sky                        73475
shower rain                      63527
light intensity shower rain      52224
mist                             32949
heavy intensity rain             19857
light intensity drizzle          16280
fog                              14763
light intensity drizzle rain      4232
very heavy rain                   3052
snow                              1458
shower sleet                      1140
overcast clouds                    240
Name: description, dtype: int64

In [4]:
# Investigating the 'main' weather feature counts
df['main'].value_counts()

Clouds     594239
Rain       414258
Clear       73475
Mist        32949
Drizzle     20512
Fog         14763
Snow         2598
Name: main, dtype: int64

In [5]:
# Encoding the weather to continious data
# Allow for the model to calculate

for col in df[['description']].columns:
    if [df['description']=='clear sky']:
        df['description'].replace('clear sky',1, inplace=True)
    if[df['description']=='few clouds']:
        df['description'].replace('few clouds', 2, inplace=True)
    if[df['description']=='scattered clouds']:
        df['description'].replace('scattered clouds',3, inplace=True)
    if [df['description']=='broken clouds']:
        df['description'].replace('broken clouds',4, inplace=True)
    if [df['description']=='fog']:
        df['description'].replace('fog',5, inplace=True)
    if [df['description']=='mist']:
        df['description'].replace('mist',6, inplace=True)
    if [df['description']=='light intensity drizzle']:
        df['description'].replace('light intensity drizzle',7, inplace=True)
    if [df['description']=='light intensity drizzle rain']:
        df['description'].replace('light intensity drizzle rain',8, inplace=True)
    if [df['description']=='light intensity shower rain']:
        df['description'].replace('light intensity shower rain',9, inplace=True)
    if [df['description']=='shower rain']:
        df['description'].replace('shower rain',10, inplace=True)
    if [df['description']=='heavy intensity rain']:
        df['description'].replace('heavy intensity rain',11, inplace=True)
    if [df['description']=='light rain']:
        df['description'].replace('light rain',12, inplace=True)
    if [df['description']=='moderate rain']:
        df['description'].replace('moderate rain',13, inplace=True)
    if [df['description']=='very heavy rain']:
        df['description'].replace('very heavy rain',14, inplace=True)
    if [df['description']=='shower sleet']:
        df['description'].replace('shower sleet',15, inplace=True)
    if [df['description']=='snow']:
        df['description'].replace('snow',16, inplace=True)
    if [df['description']=='overcast clouds']:
        df['description'].replace('overcast clouds',17, inplace=True)

In [6]:
# Ensuring the replacment was successful
df['description'].value_counts()

4     332210
12    195687
3     141655
2     120134
13     79911
1      73475
10     63527
9      52224
6      32949
11     19857
7      16280
5      14763
8       4232
14      3052
16      1458
15      1140
17       240
Name: description, dtype: int64

In [7]:
# Changing 'last_update' to a datetime type
df['last_update'] = df['last_update'].astype('datetime64[ns]')

In [8]:
# Creating a week column
df['week'] = df['last_update'].dt.week

In [9]:
# Creating a day column
df['day'] = df['last_update'].dt.weekday

In [10]:
# Investigating the feature types
df.dtypes

number                       int64
name                        object
status                      object
bike_stands                  int64
available_bikes              int64
available_stands             int64
last_update         datetime64[ns]
banking                      int64
weather_id                   int64
main                        object
description                  int64
temperature                float64
humidity                   float64
wind_speed                 float64
week                         int64
day                          int64
dtype: object

In [11]:
# Rounding the last_update to 30mins
df['last_update'] = df['last_update'].dt.round('30min') 

In [12]:
# Creating a column with type String that holds the time rounded time
df['time'] = df.last_update.map(lambda t: t.strftime('%H%M'))

In [13]:
# Dropping all duplicates
df = df.drop_duplicates()

In [14]:
# Investigating the feature types
df.dtypes

number                       int64
name                        object
status                      object
bike_stands                  int64
available_bikes              int64
available_stands             int64
last_update         datetime64[ns]
banking                      int64
weather_id                   int64
main                        object
description                  int64
temperature                float64
humidity                   float64
wind_speed                 float64
week                         int64
day                          int64
time                        object
dtype: object

In [15]:
# Dropping features that are not needed for the model
df = df.drop(['last_update', 'banking', 'bike_stands', 'weather_id', 'main', 'temperature', 'humidity', 'wind_speed'] , axis=1)

In [16]:
# Isolating the day and station to investigate
df = df.loc[(df['day'] == 4) & (df['name'] == 'GRAND CANAL DOCK')]

In [17]:
df.reset_index(drop=True)

,number,name,status,available_bikes,available_stands,description,week,day,time
0,69,GRAND CANAL DOCK,OPEN,10,30,4,8,4,0000
1,69,GRAND CANAL DOCK,OPEN,10,30,3,8,4,0030
2,69,GRAND CANAL DOCK,OPEN,9,31,3,8,4,0030
3,69,GRAND CANAL DOCK,OPEN,9,31,3,8,4,0100
4,69,GRAND CANAL DOCK,OPEN,9,31,4,8,4,0130
5,69,GRAND CANAL DOCK,OPEN,9,31,4,8,4,0200
6,69,GRAND CANAL DOCK,OPEN,9,31,4,8,4,0230
7,69,GRAND CANAL DOCK,OPEN,9,31,4,8,4,0300
8,69,GRAND CANAL DOCK,OPEN,9,31,4,8,4,0330
9,69,GRAND CANAL DOCK,OPEN,9,31,4,8,4,0400


In [18]:
# Investigating the feature types
df.dtypes

number               int64
name                object
status              object
available_bikes      int64
available_stands     int64
description          int64
week                 int64
day                  int64
time                object
dtype: object

## Preparing the columns

Here we prepare the feautres and target feature for the model. Because we have already isloated the location and day, we are limiting the 'X' features to 'time' and weather'. The target feature is 'available_bikes'

In [19]:
item_list = list(range(df.shape[1]))
list_to_remove = [0,1,2,3,4,6,7]
final_list= list(set(item_list).difference(set(list_to_remove)))
print(final_list)

[8, 5]


In [20]:
X = df.iloc[:, final_list].values
y = df.iloc[:, 3].values

## Splitting to train and test set & Creating Model

We split the X and y features into a train and test dataset. We then build a randomForest using the build in RandomForestRegression provided through scikit learn.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=False) 

In [22]:
# Running the X_train and y_train on the RandomForestRegressor
model = RandomForestRegressor(n_estimators=1000, random_state=0)  
model.fit(X_train, y_train)  

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

## Testing the model on X_train

We check the accuracy of the model on the train dataset. Then compare the predicited values to the actual values that were used to train the model.

In [23]:
y_pred = model.predict(X_train)  

In [24]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred))  
print('MSE:', metrics.mean_squared_error(y_train, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred)))
print('R2: ', metrics.r2_score(y_train,y_pred))

MAE: 3.5248080791870087
MSE: 31.51629288311596
RMSE: 5.613937377911866
R2:  0.8418273040077585


## Testing the model on X_test

We check the accuracy of the model on the test dataset. Then compare the predicited values to the test values. Analysis is provided below that represents the accuracy of the model. The train and test predicitions both produced a Mean Absolute Error of around 3.5 bikes. Our R2 is also quite accurate at 0.82. 

In [25]:
y_pred = model.predict(X_test)  

In [26]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test,y_pred))

MAE: 3.647628287865207
MSE: 33.26204045101979
RMSE: 5.767325242347599
R2:  0.8238834260800865


## Visualising

This provides a .png of the tree. It is not the whole tree but gives a good overview of the workings of a random forest

In [27]:
labels = np.array(df['available_bikes'])

In [28]:
df.columns

Index(['number', 'name', 'status', 'available_bikes', 'available_stands',
       'description', 'week', 'day', 'time'],
      dtype='object')

In [29]:
df = df.drop(['number', 'name', 'status', 'available_bikes', 'available_stands', 'week', 'day'], axis=1)

In [30]:
feature_list = list(df)

In [31]:
tree = model.estimators_[5]

In [32]:
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

In [33]:
(graph, ) = pydot.graph_from_dot_file('tree.dot')

In [34]:
graph.write_png('tree.png')

## Cross-validation testing

We used k-Fold Cross-Validation. Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample. We used the a 10-fold, denoted as cv=10. This gives a fairer indication ofhte acciracy of our model. The MAE has slightly worsened to 4.7, but in our opinion is still a good score. Howeever, the R2 has reduced significantly to 0.72. Cross-validation is a necessity in order to get a deeper understanding of the model. Nothwithstanding the deteriation in the analysis figures, we are still happy that the model is working well.

In [35]:
scoring = {'abs_error': 'neg_mean_absolute_error', 'squared_error': 'neg_mean_squared_error', 'r2':'r2'}

scores = cross_validate(model,X_test, y_test, cv=10, scoring=scoring, return_train_score=True)
print("MAE :" , abs(scores['test_abs_error'].mean()), "RMSE :", math.sqrt(abs(scores['test_squared_error'].mean())), "| r2 :", scores['test_r2'].mean())

MAE : 4.741605808903289 RMSE : 7.218286975989077 | r2 : 0.7242457015812943


In [36]:
# Looking at the co-efficient importance for each feature
print("Features and co-efficients: " , list(zip(['time','weather'], model.feature_importances_)))

Features and co-efficients:  [('time', 0.8518779844616836), ('weather', 0.1481220155383179)]


## Pickle the model

Saving the model in a .sav file for future use.

In [37]:
pickle_model = 'GRAND_CANAL_STATION_model.sav'

In [38]:
pickle.dump(model, open(pickle_model, 'wb'))

In [39]:
# loaded_model = pickle.load(open(pickle_model, 'rb'))
# result = loaded_model.score(pass_test_here)
# print(result)

## Testing Model

In [40]:
model.predict([['1800', 1]])

array([1.07339285])